# Data Science Capstone Wk. 4
For IBM Data Science Professional Certificate: Data Science Capstone course.
**Eduardo M. Pérez Rodríguez**

## Study of venues for office land acquisiton in the city of Madrid, Spain.

In this .ipynb file, we are making the analysis to help our client to choose the best area to open a new office space in the city of Madrid.

## Preprocessing and data wrangling

Let's start obtaining the data of the different areas we can divide Madrid's city, in order to choose the most suitable for this study.

First, we are obtaining the data from the Madrid's municipality data bank (https://datos.madrid.es/). We are managing the data of _"Callejero. Información adicional asociada. "_, meaning "Street maps. Aditional information related" in English. The file which we are extracting the data is a .CSV type.

In [1]:
!pip install geocoder
!pip install requests

     |████████████████████████████████| 102kB 4.4MB/s 


In [2]:
import pandas as pd
data=pd.read_csv('https://datos.madrid.es/egob/catalogo/200075-1-callejero.csv', encoding='iso-8859-1', sep=';')
data.head()


,Codigo de numero,Codigo de via,Clase de la via,Partícula de la vía,Nombre de la vía,Literal de numeracion,Codigo de distrito,Nombre del distrito,Codigo de barrio,Nombre del barrio,Seccion censal,Codigo postal,Seccion de carteria,Zona Servicio Estacionamiento Regulado,Categoria fiscal,Direccion completa para el numero,Coordenada X (Guia Urbana) cm,Coordenada Y (Guia Urbana) cm,Longitud en S R ETRS89 WGS84,Latitud en S R ETRS89 WGS84,Tipo de la via a la que pertenece el numero,Situacion de la via respecto al terreno,Tipo de denominacion de la via,Parcela catastral del numero,Tipologia del numero,Zona de valor del numero
0,31031089,31001337,AUTOVIA,,A-1,KM.001000EN,8,FUENCARRAL-EL PARDO,6,VALVERDE,166,28050,ND,0,9,"AUTOV A-1, 001000 EN ...",44305633,448250340,3º40'23.6'' W,40º29'21.82'' N,Topónimo,Nivel,Admon,,Parcela,R21S
1,31031088,31001337,AUTOVIA,,A-1,KM.001000SA,16,HORTALEZA,6,VALDEFUENTES,119,28050,ND,0,9,"AUTOV A-1, 001000 SA ...",44312246,448249077,3º40'20.75'' W,40º29'21.45'' N,Topónimo,Nivel,Admon,,Parcela,R21N
2,31031091,31001337,AUTOVIA,,A-1,KM.001100EN,8,FUENCARRAL-EL PARDO,6,VALVERDE,171,28050,ND,0,9,"AUTOV A-1, 001100 EN ...",44367522,448330933,3º39'57.57'' W,40º29'48.11'' N,Topónimo,Nivel,Admon,,Parcela,R21S
3,31031090,31001337,AUTOVIA,,A-1,KM.001100SA,16,HORTALEZA,6,VALDEFUENTES,125,28050,ND,0,9,"AUTOV A-1, 001100 SA ...",44373755,448325151,3º39'54.87'' W,40º29'46.28'' N,Topónimo,Nivel,Admon,,Parcela,R21R
4,31031093,31001337,AUTOVIA,,A-1,KM.001200EN,8,FUENCARRAL-EL PARDO,6,VALVERDE,167,28050,ND,0,9,"AUTOV A-1, 001200 EN ...",44425207,448408185,3º39'33.3'' W,40º30'13.32'' N,Topónimo,Nivel,Admon,,Parcela,R21O


In [3]:
data.shape

(206866, 26)

Now we will check the data types in the dataframe:

In [4]:
data.dtypes

Codigo de numero                                int64
Codigo de via                                   int64
Clase de la via                                object
Partícula de la vía                            object
Nombre de la vía                               object
Literal de numeracion                          object
Codigo de distrito                              int64
Nombre del distrito                            object
Codigo de barrio                                int64
Nombre del barrio                              object
Seccion censal                                  int64
Codigo postal                                   int64
Seccion de carteria                            object
Zona Servicio Estacionamiento Regulado          int64
Categoria fiscal                                int64
Direccion completa para el numero              object
Coordenada X (Guia Urbana) cm                   int64
Coordenada Y (Guia Urbana) cm                   int64
Longitud en S R  ETRS89 WGS8

Let's turn then to string in order to make then easier to manage with the dataframes:

In [5]:
data=data.astype(str)
data.dtypes

Codigo de numero                               object
Codigo de via                                  object
Clase de la via                                object
Partícula de la vía                            object
Nombre de la vía                               object
Literal de numeracion                          object
Codigo de distrito                             object
Nombre del distrito                            object
Codigo de barrio                               object
Nombre del barrio                              object
Seccion censal                                 object
Codigo postal                                  object
Seccion de carteria                            object
Zona Servicio Estacionamiento Regulado         object
Categoria fiscal                               object
Direccion completa para el numero              object
Coordenada X (Guia Urbana) cm                  object
Coordenada Y (Guia Urbana) cm                  object
Longitud en S R  ETRS89 WGS8

We need to check if there are any null or zero values:

In [6]:
data.isnull().values.any()

False

In [7]:
(data == '0').astype(int).sum(axis=0)

Codigo de numero                                    0
Codigo de via                                       0
Clase de la via                                     0
Partícula de la vía                                 0
Nombre de la vía                                    0
Literal de numeracion                               0
Codigo de distrito                                  0
Nombre del distrito                                 0
Codigo de barrio                                    0
Nombre del barrio                                   0
Seccion censal                                      0
Codigo postal                                       3
Seccion de carteria                                 0
Zona Servicio Estacionamiento Regulado         151154
Categoria fiscal                                    5
Direccion completa para el numero                   0
Coordenada X (Guia Urbana) cm                       0
Coordenada Y (Guia Urbana) cm                       0
Longitud en S R  ETRS89 WGS8

This data do not affect our study since we only need neighborhood, latitude and longitude data (Nombre del barrio, Longitud and Latitud in our dataframe).


Now, in order to print the coordinates later, we need to turn longitude and latitude from DMS format (Degrees, minutes, seconds) to decimal format. We will use the **dms2dd** function for this purpose:

In [0]:
import re

def dms2dd(s):
    # example: s = """0°51'56.29"S""" 40º29'21.45"N
    degrees, minutes, seconds, direction = re.split('[°\'\"]+', s)
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction in ('S','W'):
        dd*= -1
    else:
        dd*=1
    return dd

Before using our function, let's avoid some undesirable characters in the dataframe:

In [9]:
data['Latitude'] = data[data.columns[pd.Series(data.columns).str.startswith('Latitud')]]
data['Longitude'] = data[data.columns[pd.Series(data.columns).str.startswith('Longitud')]]
data['Latitude'] = data['Latitude'].str.replace("''", '"').str.replace(" ","").str.replace(".",".").str.replace("º", "°")
data['Longitude'] = data['Longitude'].str.replace("''", '"').str.replace(" ","").str.replace(".",".").str.replace("º", "°")
data['Latitude'] = data['Latitude'].apply(dms2dd)
data['Longitude'] = data['Longitude'].apply(dms2dd)
data.head()

,Codigo de numero,Codigo de via,Clase de la via,Partícula de la vía,Nombre de la vía,Literal de numeracion,Codigo de distrito,Nombre del distrito,Codigo de barrio,Nombre del barrio,Seccion censal,Codigo postal,Seccion de carteria,Zona Servicio Estacionamiento Regulado,Categoria fiscal,Direccion completa para el numero,Coordenada X (Guia Urbana) cm,Coordenada Y (Guia Urbana) cm,Longitud en S R ETRS89 WGS84,Latitud en S R ETRS89 WGS84,Tipo de la via a la que pertenece el numero,Situacion de la via respecto al terreno,Tipo de denominacion de la via,Parcela catastral del numero,Tipologia del numero,Zona de valor del numero,Latitude,Longitude
0,31031089,31001337,AUTOVIA,,A-1,KM.001000EN,8,FUENCARRAL-EL PARDO,6,VALVERDE,166,28050,ND,0,9,"AUTOV A-1, 001000 EN ...",44305633,448250340,3º40'23.6'' W,40º29'21.82'' N,Topónimo,Nivel,Admon,,Parcela,R21S,40.489394,-3.673222
1,31031088,31001337,AUTOVIA,,A-1,KM.001000SA,16,HORTALEZA,6,VALDEFUENTES,119,28050,ND,0,9,"AUTOV A-1, 001000 SA ...",44312246,448249077,3º40'20.75'' W,40º29'21.45'' N,Topónimo,Nivel,Admon,,Parcela,R21N,40.489292,-3.672431
2,31031091,31001337,AUTOVIA,,A-1,KM.001100EN,8,FUENCARRAL-EL PARDO,6,VALVERDE,171,28050,ND,0,9,"AUTOV A-1, 001100 EN ...",44367522,448330933,3º39'57.57'' W,40º29'48.11'' N,Topónimo,Nivel,Admon,,Parcela,R21S,40.496697,-3.665992
3,31031090,31001337,AUTOVIA,,A-1,KM.001100SA,16,HORTALEZA,6,VALDEFUENTES,125,28050,ND,0,9,"AUTOV A-1, 001100 SA ...",44373755,448325151,3º39'54.87'' W,40º29'46.28'' N,Topónimo,Nivel,Admon,,Parcela,R21R,40.496189,-3.665242
4,31031093,31001337,AUTOVIA,,A-1,KM.001200EN,8,FUENCARRAL-EL PARDO,6,VALVERDE,167,28050,ND,0,9,"AUTOV A-1, 001200 EN ...",44425207,448408185,3º39'33.3'' W,40º30'13.32'' N,Topónimo,Nivel,Admon,,Parcela,R21O,40.503700,-3.659250


As mentioned previously, we only need neighborhood, latitude and longitude data (Nombre del barrio, Longitud and Latitud) in our dataframe.

In [10]:
df=pd.DataFrame(data, columns=['Nombre del barrio', 'Latitude', 'Longitude'])
df.columns=['Neighborhood', 'Latitude', 'Longitude']
df.dtypes

Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

Since we have several latitudes and longitudes for every single address, we will use the mean latitude and logitud for each neighborhood to get a good approximation of its location.

In [11]:
df_geo=df.groupby('Neighborhood')['Latitude', 'Longitude'].mean()
df_geo.head()


,Latitude,Longitude
Neighborhood,,
ABRANTES,40.380771,-3.728200
ACACIAS,40.401900,-3.706246
ADELFAS,40.401066,-3.671138
AEROPUERTO,40.454756,-3.571151
ALAMEDA DE OSUNA,40.456939,-3.590116


In [12]:
df_Madrid=df_geo.drop_duplicates().reset_index()
df_Madrid.head()

,Neighborhood,Latitude,Longitude
0,ABRANTES,40.380771,-3.728200
1,ACACIAS,40.401900,-3.706246
2,ADELFAS,40.401066,-3.671138
3,AEROPUERTO,40.454756,-3.571151
4,ALAMEDA DE OSUNA,40.456939,-3.590116


Let's confirm that we have 131 different neighborhoods for our study:

In [13]:
df_Madrid.shape

(131, 3)

Now we need to get the coordinates of a geographically central area in Madrid. We will use the location of 'Nuevos Ministerios' intermodal station for that purpose, getting the latitude and longitud from Geocoder via OpenStreetMaps.

In [0]:
import geocoder
g = geocoder.osm('Nuevos Ministerios, Madrid')
Longitude=g.x
Latitude=g.y

In order to print the map with all the neighborhoods, we will use the **folium** library.

In [15]:
import folium



# create map of Madrid using latitude and longitude values
Madrid_map = folium.Map(location=[Latitude, Longitude], zoom_start=11)

# add markers to map
for lat, long, neigh in zip(df_Madrid['Latitude'], df_Madrid['Longitude'], df_Madrid['Neighborhood']):
    label = "{}".format(neigh )
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Madrid_map) 
    
Madrid_map

The density of the points in the map is variable. We will just keep it in mind since we know these data are official from Madrid municipality.

## Explore neighborhoods using Foursquare API

After obtaining the maps, we are now obtaining the data from the **Foursquare API** to score the most valuable venues.

Related to our problem, we will focus later in the following:
* Metro Stations
* Restaurants
* Hotels

We need Foursquare developer credentials to get CLIENT_ID, CLIENT_SECRET and VERSION to query the data.

In [0]:
CLIENT_ID = '**********************' # your Foursquare ID
CLIENT_SECRET = '******************' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Now we will use the **getNearbyVenues** function for extracting nearby venues data around 500 meters of each coordinate:

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
# type your answer here
import requests

LIMIT=100
radius=500

Madrid_venues = getNearbyVenues(names=df_Madrid['Neighborhood'],
                                   latitudes=df_Madrid['Latitude'],
                                   longitudes=df_Madrid['Longitude']
                                  )

ABRANTES            
ACACIAS             
ADELFAS             
AEROPUERTO          
ALAMEDA DE OSUNA    
ALMAGRO             
ALMENARA            
ALMENDRALES         
ALUCHE              
AMPOSTA             
APOSTOL SANTIAGO    
ARAPILES            
ARAVACA             
ARCOS               
ARGUELLES           
ATALAYA             
ATOCHA              
BELLAS VISTAS       
BERRUGUETE          
BUENAVISTA          
BUTARQUE            
CAMPAMENTO          
CANILLAS            
CANILLEJAS          
CASA DE CAMPO       
CASCO H.BARAJAS     
CASCO H.VALLECAS    
CASCO H.VICALVARO   
CASTELLANA          
CASTILLA            
CASTILLEJOS         
CHOPERA             
CIUDAD JARDIN       
CIUDAD UNIVERSITARIA
COLINA              
COMILLAS            
CONCEPCION          
CORRALEJOS          
CORTES              
COSTILLARES         
CUATRO CAMINOS      
CUATRO VIENTOS      
DELICIAS            
EL CAÑAVERAL        
EL GOLOSO           
EL PARDO            
EL PILAR            
EL PLANTIO   

In [19]:
Madrid_venues.shape

(3585, 7)

Let's check if there are any null values:

In [20]:
Madrid_venues.isnull().values.any()

False

Let's check the top ten neighborhoods with a larger amount of venues nearby:

In [21]:
Madrid_venues[['Neighborhood', 'Venue']].groupby('Neighborhood').count().sort_values(['Venue'], ascending=False).head(10)

,Venue
Neighborhood,
UNIVERSIDAD,100
CORTES,100
JUSTICIA,100
SOL,100
EMBAJADORES,100
CASTELLANA,100
RECOLETOS,100
GAZTAMBIDE,100
ALMAGRO,100


Now we want to know the number of different categories in Madrid:

In [22]:
print('There are {} uniques categories in Madrid.'.format(len(Madrid_venues['Venue Category'].unique())))

There are 264 uniques categories in Madrid.


## Analyzing each neighborhood

Now we are analyzing each neighborhood via one-hot encoding:

In [23]:
# one hot encoding
Madrid_onehot = pd.get_dummies(Madrid_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Madrid_onehot['Neighborhood'] = Madrid_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Madrid_onehot.columns[-1]] + list(Madrid_onehot.columns[:-1])
Madrid_onehot = Madrid_onehot[fixed_columns]

Madrid_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,African Restaurant,Airport,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Boarding House,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,...,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Student Center,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Trade School,Trail,Train Station,Travel Lounge,Used Bookstore,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
Madrid_onehot.shape

(3585, 264)

Let's group them by mean for each neighborhood:

In [25]:
Madrid_grouped = Madrid_onehot.groupby('Neighborhood').mean().reset_index()
Madrid_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,African Restaurant,Airport,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Boarding House,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,...,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Student Center,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Trade School,Trail,Train Station,Travel Lounge,Used Bookstore,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,ABRANTES,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.250000,0.000000,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
1,ACACIAS,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.015152,0.0,0.0,0.0,0.0,0.030303,0.075758,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.000000,0.0,0.0,0.015152,0.0,0.0,0.0,0.0,0.015152,0.000000,0.00,0.0,...,0.000000,0.0,0.015152,0.000000,0.0,0.0,0.0,0.00,0.121212,0.015152,0.0,0.0,0.0,0.0,0.0,0.045455,0.015152,0.0,0.030303,0.0,0.00,0.00,0.015152,0.0,0.0,0.0,0.0,0.0,0.0,0.015152,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,ADELFAS,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.027027,0.0,0.0,0.0,0.0,0.027027,0.054054,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.027027,0.027027,0.00,0.0,...,0.000000,0.0,0.000000,0.027027,0.0,0.0,0.0,0.00,0.054054,0.000000,0.0,0.0,0.0,0.0,0.0,0.054054,0.000000,0.0,0.054054,0.0,0.00,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,ALAMEDA DE OSUNA,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.041667,0.041667,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.041667,0.0,0.0,0.041667,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,...,0.041667,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.041667,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.083333,0.0,0.00,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,ALMAGRO,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.01,0.01,0.0,0.000000,0.0,0.0,0.0,0.0,0.020000,0.070000,0.0,0.0,0.0,0.0,0.01,0.01,0.0,0.0,0.000000,0.0,0.0,0.020000,0.0,0.0,0.0,0.0,0.010000,0.010000,0.01,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.01,0.160000,0.000000,0.0,0.0,0.0,0.0,0.0,0.010000,0.010000,0.0,0.010000,0.0,0.01,0.01,0.010000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0


In [26]:
Madrid_grouped.shape

(128, 264)

We will now get the top ten venues for each neighborhood into a dataframe:

In [27]:
num_top_venues = 10

for hood in Madrid_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Madrid_grouped[Madrid_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ABRANTES            ----
                     venue  freq
0                   Bakery  0.25
1              Pizza Place  0.25
2     Fast Food Restaurant  0.25
3               Restaurant  0.25
4             Noodle House  0.00
5                Nightclub  0.00
6  North Indian Restaurant  0.00
7                   Office  0.00
8              Opera House  0.00
9                   Palace  0.00


----ACACIAS             ----
                venue  freq
0  Spanish Restaurant  0.12
1                 Bar  0.08
2                Park  0.06
3         Supermarket  0.05
4                Café  0.05
5         Pizza Place  0.05
6       Grocery Store  0.03
7          Restaurant  0.03
8   Food & Drink Shop  0.03
9              Bakery  0.03


----ADELFAS             ----
                  venue  freq
0         Grocery Store  0.08
1                 Diner  0.05
2           Supermarket  0.05
3  Fast Food Restaurant  0.05
4                   Bar  0.05
5    Spanish Restaurant  0.05
6      Tapas Restaurant  0.0

Now we will check the most common only:

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Madrid_grouped['Neighborhood']

for ind in np.arange(Madrid_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Madrid_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(20)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ABRANTES,Pizza Place,Fast Food Restaurant,Restaurant,Bakery,Women's Store,Fish & Chips Shop,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
1,ACACIAS,Spanish Restaurant,Bar,Park,Café,Supermarket,Pizza Place,Grocery Store,Restaurant,Food & Drink Shop,Gym
2,ADELFAS,Grocery Store,Supermarket,Diner,Spanish Restaurant,Tapas Restaurant,Fast Food Restaurant,Bar,Breakfast Spot,Football Stadium,Brewery
3,ALAMEDA DE OSUNA,Plaza,Tapas Restaurant,Hobby Shop,Metro Station,Bar,Bakery,Cocktail Bar,Fried Chicken Joint,Chinese Restaurant,Bistro
4,ALMAGRO,Spanish Restaurant,Restaurant,Bar,Italian Restaurant,Mediterranean Restaurant,Japanese Restaurant,French Restaurant,Plaza,Coffee Shop,Café
5,ALMENARA,Hotel,Gym / Fitness Center,Restaurant,Chinese Restaurant,Flea Market,Spanish Restaurant,BBQ Joint,Library,Frozen Yogurt Shop,Japanese Restaurant
6,ALMENDRALES,Spanish Restaurant,Grocery Store,Chinese Restaurant,Gym,Restaurant,Seafood Restaurant,Nightclub,Noodle House,Metro Station,Fast Food Restaurant
7,ALUCHE,Bar,Italian Restaurant,Pie Shop,Park,Brewery,Café,Paella Restaurant,Bus Station,Farmers Market,Pizza Place
8,AMPOSTA,Park,Snack Place,Metro Station,Food,Spanish Restaurant,Music Venue,Supermarket,Gas Station,Farmers Market,Fast Food Restaurant
9,APOSTOL SANTIAGO,Grocery Store,Food Truck,Pharmacy,Mediterranean Restaurant,Park,Metro Station,Bakery,Flea Market,Fish Market,Fabric Shop


In [30]:
neighborhoods_venues_sorted.shape

(128, 11)

Not every neighborhood could get venues. Since some neighborhoods are restricted areas (military or aerospatial) or places under construction of the city of Madrid, we will not make changes in our analysis.

## Clustering neighborhoods:

We will use **K-Means** algorithm using **sklearn** libraryto cluster the neighborhoods by similarities between them:

In [0]:
from sklearn.cluster import KMeans

In [32]:
# set number of clusters
kclusters = 10

Madrid_grouped_clustering = Madrid_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Madrid_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 8, 8, 8, 1, 1, 7, 8, 8, 7], dtype=int32)

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Madrid_merged = df_geo

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Madrid_merged = Madrid_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Madrid_merged.head() # check the last columns!

,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Neighborhood,,,,,,,,,,,,,
ABRANTES,40.380771,-3.728200,4.0,Pizza Place,Fast Food Restaurant,Restaurant,Bakery,Women's Store,Fish & Chips Shop,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
ACACIAS,40.401900,-3.706246,8.0,Spanish Restaurant,Bar,Park,Café,Supermarket,Pizza Place,Grocery Store,Restaurant,Food & Drink Shop,Gym
ADELFAS,40.401066,-3.671138,8.0,Grocery Store,Supermarket,Diner,Spanish Restaurant,Tapas Restaurant,Fast Food Restaurant,Bar,Breakfast Spot,Football Stadium,Brewery
AEROPUERTO,40.454756,-3.571151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALAMEDA DE OSUNA,40.456939,-3.590116,8.0,Plaza,Tapas Restaurant,Hobby Shop,Metro Station,Bar,Bakery,Cocktail Bar,Fried Chicken Joint,Chinese Restaurant,Bistro


Let's look for null values before going on:

In [34]:
(Madrid_merged.isnull()).astype(int).sum(axis=0)

Latitude                  0
Longitude                 0
Cluster Labels            3
1st Most Common Venue     3
2nd Most Common Venue     3
3rd Most Common Venue     3
4th Most Common Venue     3
5th Most Common Venue     3
6th Most Common Venue     3
7th Most Common Venue     3
8th Most Common Venue     3
9th Most Common Venue     3
10th Most Common Venue    3
dtype: int64

In [35]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'].isnull()].reset_index()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AEROPUERTO,40.454756,-3.571151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EL CAÑAVERAL,40.401892,-3.554377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EL GOLOSO,40.516068,-3.697363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


These neighborhoods can be removed since there are no venues returned on them. They actually belong to reserved spaces, military bases or areas in construction, so it makes sense to skip the from this study.

In [36]:
Madrid_merged = Madrid_merged[pd.notnull(Madrid_merged['Cluster Labels'])]
(Madrid_merged.isnull()).astype(int).sum(axis=0)

Latitude                  0
Longitude                 0
Cluster Labels            0
1st Most Common Venue     0
2nd Most Common Venue     0
3rd Most Common Venue     0
4th Most Common Venue     0
5th Most Common Venue     0
6th Most Common Venue     0
7th Most Common Venue     0
8th Most Common Venue     0
9th Most Common Venue     0
10th Most Common Venue    0
dtype: int64

In [37]:
Madrid_merged=Madrid_merged.reset_index()
Madrid_merged.dtypes

Neighborhood               object
Latitude                  float64
Longitude                 float64
Cluster Labels            float64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [38]:
Madrid_merged.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ABRANTES,40.380771,-3.728200,4.0,Pizza Place,Fast Food Restaurant,Restaurant,Bakery,Women's Store,Fish & Chips Shop,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
1,ACACIAS,40.401900,-3.706246,8.0,Spanish Restaurant,Bar,Park,Café,Supermarket,Pizza Place,Grocery Store,Restaurant,Food & Drink Shop,Gym
2,ADELFAS,40.401066,-3.671138,8.0,Grocery Store,Supermarket,Diner,Spanish Restaurant,Tapas Restaurant,Fast Food Restaurant,Bar,Breakfast Spot,Football Stadium,Brewery
3,ALAMEDA DE OSUNA,40.456939,-3.590116,8.0,Plaza,Tapas Restaurant,Hobby Shop,Metro Station,Bar,Bakery,Cocktail Bar,Fried Chicken Joint,Chinese Restaurant,Bistro
4,ALMAGRO,40.432932,-3.694264,1.0,Spanish Restaurant,Restaurant,Bar,Italian Restaurant,Mediterranean Restaurant,Japanese Restaurant,French Restaurant,Plaza,Coffee Shop,Café


We will now plot a map using **folium** library. We are painting every cluster in a different color:

In [39]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests

# create map
map_clusters = folium.Map(location=[Latitude, Longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Madrid_merged['Latitude'], Madrid_merged['Longitude'], Madrid_merged['Neighborhood'], Madrid_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Detail of clusters:

It will be useful to get the detail of each cluster separately as it follows:

#### Cluster No. 0

In [40]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 0, Madrid_merged.columns[[0] + list(range(4, Madrid_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,EL PLANTIO,Shop & Service,Paella Restaurant,Brewery,Farm,Women's Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market


#### Cluster No. 1

In [41]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 1, Madrid_merged.columns[[0] + list(range(4, Madrid_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,ALMAGRO,Spanish Restaurant,Restaurant,Bar,Italian Restaurant,Mediterranean Restaurant,Japanese Restaurant,French Restaurant,Plaza,Coffee Shop,Café
5,ALMENARA,Hotel,Gym / Fitness Center,Restaurant,Chinese Restaurant,Flea Market,Spanish Restaurant,BBQ Joint,Library,Frozen Yogurt Shop,Japanese Restaurant
13,ARGUELLES,Tapas Restaurant,Spanish Restaurant,Hotel,Mexican Restaurant,Park,Supermarket,Mediterranean Restaurant,Gastropub,Deli / Bodega,Burger Joint
14,ATALAYA,Spanish Restaurant,Bakery,Restaurant,Paella Restaurant,American Restaurant,Gym Pool,Chinese Restaurant,Farmers Market,Fabric Shop,Falafel Restaurant
16,BELLAS VISTAS,Spanish Restaurant,Seafood Restaurant,Bakery,Grocery Store,Snack Place,Donut Shop,Breakfast Spot,Fast Food Restaurant,Frozen Yogurt Shop,Steakhouse
20,CAMPAMENTO,Bus Station,Light Rail Station,Spanish Restaurant,Metro Station,Women's Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market
24,CASCO H.BARAJAS,Hotel,Spanish Restaurant,Restaurant,Gastropub,Tapas Restaurant,Flea Market,Bistro,Fast Food Restaurant,Boarding House,Coffee Shop
27,CASTELLANA,Spanish Restaurant,Restaurant,Coffee Shop,Boutique,Bistro,Cocktail Bar,Japanese Restaurant,Mediterranean Restaurant,Tapas Restaurant,Furniture / Home Store
29,CASTILLEJOS,Spanish Restaurant,Restaurant,Café,Coffee Shop,Tapas Restaurant,Cocktail Bar,Hotel,Bar,Chinese Restaurant,Salad Place
33,COLINA,Chinese Restaurant,Spanish Restaurant,Bar,Cupcake Shop,Paper / Office Supplies Store,Restaurant,Mediterranean Restaurant,Bakery,Thai Restaurant,Fast Food Restaurant


#### Cluster No. 2:

In [42]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 2, Madrid_merged.columns[[0] + list(range(4, Madrid_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,PIOVERA,Theater,Cheese Shop,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Exhibit,Fish Market


#### Cluster No. 3:

In [43]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 3, Madrid_merged.columns[[0] + list(range(4, Madrid_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
72,MARROQUINA,Park,Brewery,Bakery,Women's Store,Fish Market,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop
117,VALDEFUENTES,Park,Women's Store,Fish Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Exhibit


#### Cluster No. 4:

In [44]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 4, Madrid_merged.columns[[0] + list(range(4, Madrid_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ABRANTES,Pizza Place,Fast Food Restaurant,Restaurant,Bakery,Women's Store,Fish & Chips Shop,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
11,ARAVACA,Italian Restaurant,Soccer Field,Athletics & Sports,Grocery Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Exhibit
40,CUATRO VIENTOS,Burger Joint,Pizza Place,Tapas Restaurant,Fast Food Restaurant,Supermarket,Fish Market,Farm,Farmers Market,Fish & Chips Shop,Women's Store
43,EL PILAR,Italian Restaurant,Fast Food Restaurant,Restaurant,Burger Joint,Tapas Restaurant,Bar,Cosmetics Shop,Coffee Shop,Chocolate Shop,Sandwich Place
71,LUCERO,Pizza Place,Grocery Store,Italian Restaurant,Supermarket,Asian Restaurant,Fast Food Restaurant,Fish & Chips Shop,Falafel Restaurant,Farm,Farmers Market
89,PEÑA GRANDE,Argentinian Restaurant,Pizza Place,Restaurant,Nightclub,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market


#### Cluster No. 5:

In [45]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 5, Madrid_merged.columns[[0] + list(range(4, Madrid_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,CASA DE CAMPO,Theater,Park,Mediterranean Restaurant,Restaurant,Bar,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Exhibit
38,COSTILLARES,Bar,Gym / Fitness Center,Grocery Store,Supermarket,Music School,Pet Store,Food,Italian Restaurant,Diner,Spanish Restaurant
95,PUEBLO NUEVO,Bar,Metro Station,Brewery,Tapas Restaurant,Gym,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Women's Store
96,PUERTA BONITA,Rest Area,Pizza Place,Music Venue,Historic Site,Bar,Women's Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
123,VENTAS,Bar,Spanish Restaurant,Restaurant,Mediterranean Restaurant,Gym / Fitness Center,Trail,Bakery,Flea Market,Pizza Place,Food


#### Cluster No. 6:

In [46]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 6, Madrid_merged.columns[[0] + list(range(4, Madrid_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,CIUDAD UNIVERSITARIA,Bar,Other Nightlife,Track,Fish & Chips Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Women's Store,Fabric Shop
92,PORTAZGO,Bar,Spanish Restaurant,Women's Store,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop


#### Cluster No. 7:

In [47]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 7, Madrid_merged.columns[[0] + list(range(4, Madrid_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,ALMENDRALES,Spanish Restaurant,Grocery Store,Chinese Restaurant,Gym,Restaurant,Seafood Restaurant,Nightclub,Noodle House,Metro Station,Fast Food Restaurant
9,APOSTOL SANTIAGO,Grocery Store,Food Truck,Pharmacy,Mediterranean Restaurant,Park,Metro Station,Bakery,Flea Market,Fish Market,Fabric Shop
18,BUENAVISTA,Music School,Metro Station,Grocery Store,BBQ Joint,Spanish Restaurant,Gym,Gym / Fitness Center,Farm,Farmers Market,Fast Food Restaurant
25,CASCO H.VALLECAS,Grocery Store,Food Truck,Basketball Court,Spanish Restaurant,Gym,Fish & Chips Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
26,CASCO H.VICALVARO,Grocery Store,Brewery,Convenience Store,Beer Bar,Café,Spanish Restaurant,Market,Pizza Place,Ice Cream Shop,Food & Drink Shop
49,ENTREVIAS,Grocery Store,Park,Gym / Fitness Center,Restaurant,Train Station,Chinese Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
53,FUENTELARREINA,Grocery Store,Hotel,Cosmetics Shop,Fast Food Restaurant,Fish Market,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fish & Chips Shop
67,LOS ANGELES,Café,Brewery,Grocery Store,Spanish Restaurant,Restaurant,Bar,Women's Store,Farm,Farmers Market,Fast Food Restaurant
70,LOS ROSALES,Metro Station,Pizza Place,Brewery,Train Station,Supermarket,Chinese Restaurant,Restaurant,Electronics Store,Pet Store,Basketball Court
74,MIRASIERRA,Grocery Store,Convenience Store,Gym,Metro Station,Sports Club,Café,Women's Store,Fast Food Restaurant,Farm,Farmers Market


#### Cluster No. 8:

In [48]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 8, Madrid_merged.columns[[0] + list(range(4, Madrid_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,ACACIAS,Spanish Restaurant,Bar,Park,Café,Supermarket,Pizza Place,Grocery Store,Restaurant,Food & Drink Shop,Gym
2,ADELFAS,Grocery Store,Supermarket,Diner,Spanish Restaurant,Tapas Restaurant,Fast Food Restaurant,Bar,Breakfast Spot,Football Stadium,Brewery
3,ALAMEDA DE OSUNA,Plaza,Tapas Restaurant,Hobby Shop,Metro Station,Bar,Bakery,Cocktail Bar,Fried Chicken Joint,Chinese Restaurant,Bistro
7,ALUCHE,Bar,Italian Restaurant,Pie Shop,Park,Brewery,Café,Paella Restaurant,Bus Station,Farmers Market,Pizza Place
8,AMPOSTA,Park,Snack Place,Metro Station,Food,Spanish Restaurant,Music Venue,Supermarket,Gas Station,Farmers Market,Fast Food Restaurant
10,ARAPILES,Spanish Restaurant,Bar,Bakery,Café,Restaurant,Burrito Place,Sandwich Place,Supermarket,Theater,Beer Bar
12,ARCOS,Multiplex,Gym / Fitness Center,Chinese Restaurant,Soccer Field,Burger Joint,Restaurant,Beer Garden,Park,Asian Restaurant,Farmers Market
15,ATOCHA,Bar,Café,Spanish Restaurant,Restaurant,Italian Restaurant,Gym,Mediterranean Restaurant,Chinese Restaurant,Sushi Restaurant,Coffee Shop
17,BERRUGUETE,Tapas Restaurant,Bar,Spanish Restaurant,Pizza Place,Gym / Fitness Center,Grocery Store,Restaurant,Frozen Yogurt Shop,Clothing Store,Supermarket
19,BUTARQUE,Mexican Restaurant,Sports Bar,Athletics & Sports,Sandwich Place,Food & Drink Shop,Food,Food Court,Flower Shop,Flea Market,Fabric Shop


#### Cluster No. 9:

In [49]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 9, Madrid_merged.columns[[0] + list(range(4, Madrid_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,EL PARDO,Gym / Fitness Center,History Museum,Women's Store,Fish Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market


## Scoring:

We will now work in the scoring. We will score each **neighborhood** first (with the venues restaurant, hotel and metro station equally weighted as requested by our client). With this results, we will score each **cluster** so we will make the choice of our client easier. 

In [0]:
Madrid_onehot_group=Madrid_onehot.groupby(['Neighborhood']).sum()

In [0]:
Madrid_onehot_group_sum=Madrid_onehot_group[['Restaurant', 'Hotel', 'Metro Station']]

We need to standarize and weight each of the venues (according to our client demand, they will be equally weighted):

In [0]:
from sklearn import preprocessing
x = Madrid_onehot_group_sum.values #returns a numpy array
scaler = preprocessing.StandardScaler()
x_scaled = scaler.fit_transform(x)
Madrid_onehot_group_sum_est = pd.DataFrame(x_scaled, index=Madrid_onehot_group_sum.index, columns=Madrid_onehot_group_sum.columns)
Madrid_onehot_group_sum_est['SCORE']= 1/3*(Madrid_onehot_group_sum_est['Restaurant'] + 1/3*Madrid_onehot_group_sum_est['Hotel'] + 1/3*Madrid_onehot_group_sum_est['Metro Station'])

## Results:

Finally we get a dataframe with the scored neighborhoods, from best to least regarding the needs of our client. Let's check the top ten:

In [64]:
Madrid_onehot_group_sum_est=Madrid_onehot_group_sum_est.sort_values(['SCORE'], ascending=False).reset_index()
Madrid_final=pd.merge(Madrid_onehot_group_sum_est, Madrid_merged[['Neighborhood', 'Cluster Labels']], on=['Neighborhood'])
Madrid_final.head(10)


,Neighborhood,Restaurant,Hotel,Metro Station,SCORE,Cluster Labels
0,RECOLETOS,4.384123,3.297158,-0.436113,1.779268,1.0
1,CORTES,2.283945,7.048591,-0.436113,1.496035,1.0
2,CASTELLANA,3.964088,0.170964,-0.436113,1.291902,1.0
3,ALMAGRO,3.544052,0.796203,-0.436113,1.221361,1.0
4,LISTA,3.124016,1.421441,-0.436113,1.150820,1.0
5,CASTILLEJOS,1.863909,1.421441,1.424635,0.937534,1.0
6,JUSTICIA,1.863909,2.671919,-0.436113,0.869726,8.0
7,TRAFALGAR,2.703980,-0.454275,-0.436113,0.802395,1.0
8,PALOS DE MOGUER,2.283945,0.796203,-0.436113,0.801325,1.0
9,SOL,1.443873,2.671919,-0.436113,0.729714,8.0


And now the worst ten:

In [68]:
Madrid_final.tail(10)

,Neighborhood,Restaurant,Hotel,Metro Station,SCORE,Cluster Labels
118,HORCAJO,-0.656306,-0.454275,-0.436113,-0.317701,1.0
119,LA PAZ,-0.656306,-0.454275,-0.436113,-0.317701,1.0
120,LOS CARMENES,-0.656306,-0.454275,-0.436113,-0.317701,8.0
121,LUCERO,-0.656306,-0.454275,-0.436113,-0.317701,4.0
122,MARROQUINA,-0.656306,-0.454275,-0.436113,-0.317701,3.0
123,ORCASITAS,-0.656306,-0.454275,-0.436113,-0.317701,7.0
124,PALOMERAS BAJAS,-0.656306,-0.454275,-0.436113,-0.317701,8.0
125,PAVONES,-0.656306,-0.454275,-0.436113,-0.317701,7.0
126,PIOVERA,-0.656306,-0.454275,-0.436113,-0.317701,2.0
127,ZOFIO,-0.656306,-0.454275,-0.436113,-0.317701,1.0


According to these data, it seems that clusters no. 1 and 8 are the most suitable for our client. Let's now aggregate all the scores for each Cluster to check this:

In [65]:
Madrid_final[['Cluster Labels', 'SCORE']].groupby(['Cluster Labels']).sum().sort_values(['SCORE'], ascending=False)

,SCORE
Cluster Labels,
1.0,7.482688
0.0,-0.317701
2.0,-0.317701
9.0,-0.317701
3.0,-0.635401
6.0,-0.635401
8.0,-0.878025
5.0,-0.894980
4.0,-1.346156


Cluster no. 1 takes advantage among all the rests, being the most suitable accordint to data.

## Conclussions

We can finally advise our client that the most suitable neighborhoods are the ones located in **cluster no. 1** according to our study.

Regarding neighborhoods in particular, after using the scoring technique, we find that the following neighborhoods seem the best for our purposes: **Recoletos, Cortes, Castellana, Almagro, Lista, Castillejos, Justicia, Trafalgar, Palos de Moguer** and **Sol.** Most of them belong to **cluster no. 1** as well.

Submited by **Eduardo Manuel Pérez Rodríguez** for **IBM Data Science Professional certificate course.** August 2019.